# Read the SQL author table - clean it and make a table and push it into database

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import *
#import nltk

In [16]:
%matplotlib inline
pd.set_option('notebook_repr_html',False)

In [17]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [18]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'opensyl'
username = 'parama'

In [19]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://parama@localhost/opensyl


# Preparing the author-gender table

In [2]:
from sklearn.externals import joblib
clf = joblib.load('randfor.pkl')

In [7]:
%pwd

u'/home/parama/Insight_Proj/codes'

In [3]:
def nametogender(aname):
    
    f1 = Lastl.encode(get_last_letter(aname))
    f2 = Seclastl.encode(get_2ndlast_letter(aname))
    f3 = Firstl.encode(get_first_letter(aname))
    f4 = Secondl.encode(get_2nd_letter(aname))
    cinput = np.concatenate([f1, f2, f3, f4], axis=1)
    return cinput

In [4]:
df = pd.read_csv('final_all_names.csv',index_col=False)
df = df.drop('Unnamed: 0',1)
df.name = df.name.str.replace('[^a-zA-Z]', '')
df['name'] = df['name'].str.lower()

length_Sel = []
for everyname in df.name:
    if len(everyname) >= 2:
        length_Sel.append(1)
    else:
        length_Sel.append(0)
        
df['lengthselector'] = length_Sel
df = df[df.lengthselector == 1]

get_last_letter = lambda x: x[-1]
get_2ndlast_letter = lambda x: x[-2]
get_first_letter = lambda x: x[0]
get_2nd_letter = lambda x: x[1]
#vowels = 'AEIOUaeiou'
#no_of_vowel = lambda x : sum( in vowels for s in x)

df['factor1'] = df.name.map(get_last_letter)
df['factor2'] = df.name.map(get_2ndlast_letter)
df['factor3'] = df.name.map(get_first_letter)
df['factor4'] = df.name.map(get_2nd_letter)

In [5]:
class Encoder(object):
    def __init__(self,tokens):
        uniquetokens = np.unique(tokens)
        self.tokendict = {uniquetoken: i for i, uniquetoken in enumerate(uniquetokens)}
        self.tokendictinv = {i: uniquetoken for i, uniquetoken in enumerate(uniquetokens)}
        self.ntokens = len(uniquetokens)
        
    def encode(self,tokens):
        n = len(tokens)
        encodearray = np.zeros((n,self.ntokens))
        for i, token in enumerate(tokens):
            tokenid = self.tokendict.get(token,None)
            if tokenid is not None:
                encodearray[i,tokenid] = 1
        return encodearray
    
    def colidx_to_token(self,colidx):
        return self.tokendictinv[colidx]

In [6]:
Lastl = Encoder(df.factor1)
Seclastl = Encoder(df.factor2)
Firstl = Encoder(df.factor3)
Secondl = Encoder(df.factor4)

In [7]:
clf.predict(nametogender('Minghua')), clf.predict_proba(nametogender('Minghua'))

(array([1]), array([[ 0.28625,  0.71375]]))

In [8]:
dfs = pd.read_csv('sql_authors.csv',index_col=False)
dfs.head(5)
dfs = dfs[~pd.isnull(dfs['authors'])]

In [25]:
clf.predict_proba(nametogender('William'))

array([[ 0.64407143,  0.35592857]])

In [ ]:
non_names = ['editor','author','Admin','Administration','Administrative','General','Publisher','Publishing',
             'Random','Lawyer','Engineer','President','Author','Editor','writer','Writer','Reader','College',
            'School','University','Institute','Institution','Government','Authority','Member','Authorities']
genders = []
for everyname in dfs.authors:
    if len(everyname) < 2 or (everyname in non_names):
        newgender = 'Unknown' 
    else:
        gresult = clf.predict(nametogender(everyname))[0]
        mprob = clf.predict_proba(nametogender(everyname))
        if gresult == 0:
            if mprob[0,0] > 0.80: 
                newgender = 'm'
            else:
                newgender = 'Unknown'
        elif gresult == 1:
            if mprob[0,1] > 0.80:
                newgender = 'f'
            else:
                newgender = 'Unknown'         
    genders.append(newgender)
dfs['gender'] = genders

In [26]:
non_names = ['editor','author','Admin','Administration','Administrative','General','Publisher','Publishing',
             'Random','Lawyer','Engineer','President','Author','Editor','writer','Writer','Reader','College',
            'School','University','Institute','Institution','Government','Authority','Member','Authorities']
genders = []
for everyname in dfs.authors:
    if len(everyname) < 2 or (everyname in non_names):
        newgender = 'Unknown' 
    else:
        gresult = clf.predict(nametogender(everyname))[0]
        mprob = clf.predict_proba(nametogender(everyname))
        if mprob[0,0] > 0.60:
            newgender = 'm'
        elif mprob[0,1] > 0.60:
            newgender = 'f'
        else:
            newgender = 'Unknown'         
    genders.append(newgender)
dfs['gender'] = genders

In [13]:
dfs.head(15)

,id,authors,surname,gender
0,15,Walter,De la Mare,m
1,24,Ernst,Heidrich,m
2,31,William,Gladstone,Unknown
3,35,Gxcxarard,Masson,m
4,41,Gustav,Heider,m
5,59,Alexander,MacGibbon,Unknown
6,78,Wilhelm,Hasbach,m
7,81,Arthur,Tanner,m
8,93,Sidney,Colvin,Unknown
9,102,Andrxcxa,Grabar,f


In [ ]:
#Useful stuff could be used later
df.ix[df.authors == 'Authorities']


In [29]:
dfs.to_sql('gender_info_prob', engine)

In [30]:
dfs.shape

(931954, 4)